In [474]:
import sklearn
# Import all of the scikit learn stuff
from __future__ import print_function
import pandas as pd
import warnings
# Suppress warnings from pandas library
warnings.filterwarnings("ignore", category=DeprecationWarning,
module="pandas", lineno=570)
import numpy as np

In [475]:
# !pip install -U textblob

In [476]:
# Import data as csv. Change directory to file location on your computer.
data = pd.read_csv("review_temp.csv", "," , ";")

In [477]:
menu_db = pd.read_csv("data/scraper_data.csv", ",")
menu_db = menu_db.drop(menu_db.columns[0], axis=1)
menu_db = menu_db.dropna(subset=['menu_item'])

In [478]:
# Extract bodies column from DataFrame
reviews = data.text.values

In [479]:
menu_db.iloc[0]

description                       NaN
id             1cl8ti5S8cJLHGetUe_s6w
menu_item                 Canned Soda
price                           $1.10
rest_name                 Bubba Phatz
Name: 0, dtype: object

In [480]:
business_list = set(data.business_id.values)
len(business_list)

4546

In [481]:
scored_list = []

In [482]:
def run_recommender():
    count = 0
    for biz_id in business_list:
        menu_list = menu_db.loc[menu_db['id'] == biz_id]
        biz_reviews = data.loc[data['business_id'] == biz_id]
        for item_name in menu_list.menu_item.values:
            item_dict = score_calculator(biz_id, item_name, biz_reviews, 0.5, 0.5)
            if item_dict:
                scored_list.append(item_dict)

In [483]:
def score_calculator(biz_id, item_name, reviews_table, w1, w2):
    item_dict = {}
    score = 0
    for i in range(reviews_table.shape[0]):
        review_row = reviews_table.iloc[i]
        matches_weight = review_contains_item(item_name, review_row['text'], 1.25)
        if matches_weight > 0:
            sentiment_score = generate_sentiment(review_row['text'])
            star_score = int(review_row['stars'])
            score += (sentiment_score*w1 + star_score*w2) * matches_weight

    item_dict['id'] = biz_id
    item_dict['menu_item'] = item_name
    item_dict['rec_rating'] = score
    return item_dict

In [484]:
def review_contains_item(item_name, review, weight):
    if item_name == None:
        return -1
    if type(item_name) != type("hello"):
        return -1
    if item_name.lower() in review.lower():
        return weight
    for word in item_name.split(" "):
        if word.lower() in review.lower():
            return 1
    return -1

In [485]:
from textblob import TextBlob as tb
def generate_sentiment(review):
    #nltk
    review_converted = tb(review)
    return 5*review_converted.sentiment.polarity

In [486]:
run_recommender()

In [487]:
df = pd.DataFrame(scored_list)

In [488]:
df.to_csv("ranking.csv", sep = ",")